#### 문장의 유사도 분석
: 두개의 문장이 비슷한 것인지 또는 관련이 있는 것인지 분석
- 레벤슈타인 거리
- N-gram
- 코사인 유사도
- 유클리안 유사도
- 맨하튼 유사도

#### 레벤슈타인 거리(Lvenshtein Distance)
- 두개의 문자열이 어느 정도 다른지를 나타내는 것으로 편집거리(Edit Distance)라고도 한다.
- 위학 분야에서는 DNA배열의 유사성을 판단할때도 사용.

In [2]:
from lvenshtein import Lvenshtein
lv = Lvenshtein()

In [3]:
# 가나다라와 가마바라의 거리
print(lv.calc_distance("가나다라", '가마바라'))

2


In [5]:
# 신촌역과 가장 근접한 순서로 정렬

samples = ['신촌역', '신천군', '신천역', '마곡역', '신발']
base = samples[0]
r = sorted(samples, key = lambda n: lv.calc_distance(base, n))

for n in r:
    print(lv.calc_distance(base, n), n)

0 신촌역
1 신천역
2 신천군
2 마곡역
2 신발


---
#### 코사인(cosine) 유사도

In [6]:
a = "딥러닝은 매우 재미있는 기술이라 공부하고 있습니다."
b = "공부하면 재미있는 기술이라 딥러닝을 배우고 있습니다."

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
sentences = (a, b)
tfid_Vectorizer = TfidfVectorizer()

In [11]:
sentences

('딥러닝은 매우 재미있는 기술이라 공부하고 있습니다.', '공부하면 재미있는 기술이라 딥러닝을 배우고 있습니다.')

In [13]:
# 문장 벡터화 하기 : 사전 만들기
tfid_matrix = tfid_Vectorizer.fit_transform(sentences)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

# 첫번째 문장과 두번째 문장 비교
cos_similar = cosine_similarity(tfid_matrix[0:1], tfid_matrix[1:2])
print("코사인 유사도 측정:", cos_similar)

코사인 유사도 측정: [[0.33609693]]


#### 유사도를 이용한 추천 시스템 구현
: 코사인 유사도만으로 영화의 줄거리에 기반하여 영화를 추천하는 시스템

In [16]:
import pandas as pd

In [18]:
data = pd.read_csv("../Data/movies_metadata.csv", low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [19]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [20]:
data[['title','overview']].head()

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...


In [21]:
# 상위 2만개
data = data.head(20000)

In [23]:
data['overview'].isnull().sum()

np.int64(135)

In [24]:
# 결측치를 빈값으로 대체
data['overview'] = data['overview'].fillna('')

In [25]:
# 행렬 크기 구하기
tfidf = TfidfVectorizer(stop_words='english')
tfid_matrix = tfidf.fit_transform(data['overview'])
tfid_matrix.shape

(20000, 47487)

In [28]:
cosine_sim = cosine_similarity(tfid_matrix, tfid_matrix)
cosine_sim.shape

(20000, 20000)

In [31]:
cosine_sim

array([[1.        , 0.01575748, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01575748, 1.        , 0.04907345, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04907345, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.08375766],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08375766, 0.        ,
        1.        ]])

In [29]:
# 기존데이터프레임으로 부터 영화 타일틀을 key, 영화의 인덱스를 value로 하는 딕셔너리
title_to_index = dict(zip(data['title'], data.index))

In [30]:
# 영화 제목을 Father of the Bride Part II의 인덱스
idx = title_to_index['Father of the Bride Part II']
idx

4

In [33]:
# 선택한 영화의 제목을 입력하면 코사인 유사도를 통해 가장 overview가 
# 유사한 10개의 영화를 찾아내는 함수

def get_recommendations(title, consine_sim=cosine_sim):
    # 선택한 영화의 타이틀로 부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]
    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))
    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]
    # 가장 유사한 10개의 영화의 인덱스를 얻는다
    movies_indices = [idx[0] for idx in sim_scores]
    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return data['title'].iloc[movies_indices]

In [37]:
# 영화 : The Dark Knight Rises 
get_recommendations("The Dark Knight Rises")

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object